In [1]:
import torch
import numpy as np
import torch.nn.functional as F
from torchkeras import Model, summary
#自动类型推导
a = torch.tensor(1)
b = torch.tensor(1.00)
c = torch.tensor(True)

print(a.dtype, b.dtype, c.dtype)
#可以看出分别的默认数据类型

torch.int64 torch.float32 torch.bool


In [2]:
#指定数据类型
a = torch.tensor(1, dtype=torch.int32)
b = torch.tensor(1.0, dtype=torch.float64)
print(a.dtype, b.dtype)

torch.int32 torch.float64


In [20]:
#使用特定类型构造函数   要用序列的形式
a = torch.IntTensor([1, 2, 3])
print(a, a.dtype)
b = torch.FloatTensor([1.2])
print(b, b.dtype)
c = torch.BoolTensor([1, 0, 1])
print(c, c.dtype)

tensor([1, 2, 3], dtype=torch.int32) torch.int32
tensor([1.2000]) torch.float32
tensor([ True, False,  True]) torch.bool


In [22]:
#不同类型进行转换
a = torch.tensor(1)
print(a, a.dtype)
b = a.float()         #使用float方法
print(b, b.dtype)
c = b.type(torch.float64)   #使用type方法
print(c, c.dtype)
d = torch.tensor([1, 2, 3]).type_as(c)#使用type_as方法转换为某个tensor同类型
print(d, d.dtype)

tensor(1) torch.int64
tensor(1.) torch.float32
tensor(1., dtype=torch.float64) torch.float64
tensor([1., 2., 3.], dtype=torch.float64) torch.float64


----

In [34]:
#torch.from_numpy() 从numpy数组得到tensor   不copy数据（一个改变另一个也改变）
arr = np.zeros(3)
tensor = torch.from_numpy(arr) #float转化为float32

print(tensor)
print("-------------")
#直接tensor也可以啊  但是它是copy数据的（一个改变另一个不变）
#Initial data for the tensor. Can be a list, tuple, NumPy ndarray, scalar, and other types.

ls = torch.tensor(arr, dtype=torch.float32)

np.add(arr, 1, out=arr)
print(arr)
print(tensor)
print(ls)


tensor([0., 0., 0.], dtype=torch.float64)
-------------
[1. 1. 1.]
tensor([1., 1., 1.], dtype=torch.float64)
tensor([0., 0., 0.])


In [38]:
#numpy方法从tensor得到numpy数组
#二者共享内存
tensor = torch.zeros(3)
num = tensor.numpy()

print(tensor)
print(num)

#通过clone（）来拷贝张量， 中断这种关系
num1 = tensor.clone().numpy()
num2 = tensor.data.numpy()  #也可以用tensor.data.numpy()

print(num1)
print(num2)

tensor([0., 0., 0.])
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]


In [41]:
#item方法和tolist方法将张量转化为Python数值和数值list
scal = torch.tensor(1)
s = scal.item()
print(s)

metrix = torch.ones((3,4))
li = metrix.tolist()
print(li)



1
[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0]]


In [8]:
#求导数
#y =a x ** 2 +bx + c
x = torch.tensor(1.0, requires_grad=True)
# x = torch.tensor(1.0)
a = torch.tensor(2.0)
b = torch.tensor(0.3)
c = torch.tensor(3.0)
y = a * torch.pow(x, 2) + b * x + c

y.backward()
print(x.grad)
print(a.grad)

tensor(4.3000)
None


In [11]:
# w = torch.tensor([[3.0, 1.0]], requires_grad=True)
w = torch.Tensor([[3.0, 1.0]])
b = torch.tensor([[3.0]], requires_grad=True)
x = torch.randn(10, 2)
y = torch.randn(10, 1)
y1 = x@w.t() + b
y_ = F.relu(y1)
loss = torch.mean(torch.pow(y-y_, 2))

loss.backward()
optimizer = torch.optim.SGD(params=[w, b],lr = 0.01)

print(w.grad)
print(b.grad)
# print(y_.grad)
# print(y1.grad)
# print("y1 data", y1.data)
# print(w.data)
# optimizer.step()
# print(w.grad)
# print(b.grad)
# print(w.data)
# print(b.data)
# print("y1 data", y1.data)
# optimizer.zero_grad()
# print(w.grad)
# print(b.grad)
# print(w.data)
# print(b.data)
# print("y1 data", y1.data)


None
tensor([[6.5866]])


In [18]:
class Net(torch.nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(n_inputs, 64)
        self.fc2 = torch.nn.Linear(64, n_outputs)
        self.logstd = torch.nn.Parameter(torch.zeros(1, n_outputs))

        self.norm_layer(self.fc1)
        self.norm_layer(self.fc2)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        y = self.logstd.expand_as(x)
        return x, y

    @staticmethod
    def norm_layer(layer, std=1.0, bias_std=0.0):
        torch.nn.init.orthogonal_(layer.weight, std)
        torch.nn.init.constant_(layer.bias, bias_std)



net = Net(3, 1)
# print(net.fc1.weight)
# print(net.fc1.bias)
print(net)
summary(net, input_shape=(3, 3))

Net(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 3, 64]             256
            Linear-2                 [-1, 3, 1]              65
Total params: 321
Trainable params: 321
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.000034
Forward/backward pass size (MB): 0.001488
Params size (MB): 0.001225
Estimated Total Size (MB): 0.002747
----------------------------------------------------------------


In [15]:
#torch.squeeze  和 torch.unsqueeze
#
m = np.array([1, 2, 3])
n = torch.from_numpy(m)
print(n.dim())
w = n.unsqueeze(1)
w.squeeze()

1


tensor([1, 2, 3])

In [20]:
a = torch.tensor([1, 2, 3])
b = a.unsqueeze(0)
c = b.data.numpy()[0]
c[0]

1

In [12]:
a = torch.Tensor(24)
b = torch.nn.parameter.Parameter(torch.zeros((1, 5)))
c = b.expand_as(torch.zeros(1, 5))
print(type(b))
print(type(c))
# print(torch.zeros((1, 5)))
# print(torch.zeros(1, 5))
# torch.zeros(1, 5)

<class 'torch.nn.parameter.Parameter'>
<class 'torch.Tensor'>


In [15]:
a = torch.tensor([[1, 2, 3], [2, 3, 4], [4, 5, 6]])
a.size()

torch.Size([3, 3])

In [21]:
a = torch.tensor([[1, 2, 3]])
b = torch.tensor([[1, 2, 3]]).view(3, 1)
torch.mul(a, b)

tensor([[1, 2, 3],
        [2, 4, 6],
        [3, 6, 9]])